In [1]:
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
df = pd.read_csv('../data/num_data.csv')

# import io
# df = pd.read_csv(io.BytesIO(uploaded['num_data.csv']))


In [5]:
def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    return rmse

### Data Processing

In [6]:
df = pd.read_csv('../data/num_data.csv')

In [7]:
POLLUTION = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

In [8]:
WEATHER = ['PM2.5', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM']

In [9]:
dataset = df

In [10]:
dataset.shape

(420768, 16)

In [11]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = int(data_size * 0.2)
valid_size = data_size - train_size - test_size

In [12]:
training_set = dataset[:train_size].iloc[:,4:16].values
valid_set = dataset[train_size:train_size+valid_size].iloc[:,4:16].values
test_set = dataset[data_size-test_size:].iloc[:,4:16].values

In [13]:
y = dataset.iloc[:,0].values
y = y.reshape(-1,1)
n_feature = training_set.shape[1]
y.shape

(420768, 1)

In [14]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [15]:
# convert dataset into sequences, where n_steps_in is the input sequence lengith, 
# and n_steps_out is the output sequence length 
def convert_to_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = [], []
    for i in range(len(sequences)):
        tail_x = i + n_steps_in
        out_tail_x = tail_x + n_steps_out-1
        if out_tail_x > len(sequences):
            break
        seq_x, seq_y = sequences[i:tail_x, :], sequences[tail_x-1:out_tail_x, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [16]:
n_steps_in = 12
n_steps_out = 12
X_train, y_train = convert_to_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = convert_to_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = convert_to_sequences(test_set_scaled, n_steps_in, n_steps_out)

## Grid Search Control 


In [17]:
n_activation = ['tanh', 'sigmoid', 'relu']
act = n_activation[1]

n_learn_rate = [0.01, 0.001, 0.0001]
lr = n_learn_rate[0]

n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
opt = n_optimizers[0]

n_epoches = [50]
epoch = n_epoches[0]

n_batch_size = [1024, 2048, 5096]
batch = n_batch_size[-1]

n_of_neurons = [10, 50, 200]
neuron = n_of_neurons[1]

INFO:plaidml:Opening device "metal_intel(r)_iris(tm)_graphics_6100.0"


In [18]:
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'epoch', 
                               'batch', 'neuron'])

In [19]:
for batch in [256]:
    for lr in n_learn_rate:
        n_optimizers = [optimizers.Adam(lr=lr)]
        for opt in n_optimizers: 
            for loss_function in ['mean_squared_error', 'mean_absolute_error']:
                LSTM_LSTM_reg = Sequential()
                LSTM_LSTM_reg.add(LSTM(units=neuron, return_sequences=True, input_shape=(X_train.shape[1],n_feature), activation=act))
                LSTM_LSTM_reg.add(LSTM(units=neuron, activation=act))
                LSTM_LSTM_reg.add(Dense(units=n_steps_out))

                LSTM_LSTM_reg.compile(optimizer=opt,loss=loss_function)


                regressor = LSTM_LSTM_reg
                model = 'LSTM_LSTM'

                print('training start for', model)    
                start = time.process_time()
                regressor.fit(X_train,y_train,epochs=epoch,batch_size=batch)
                train_time = round(time.process_time() - start, 2)

                print('results for training set')
                y_train_pred = regressor.predict(X_train)
                train_rmse = return_rmse(y_train,y_train_pred)

                print('results for valid set')
                y_valid_pred = regressor.predict(X_valid)
                valid_rmse = return_rmse(y_valid,y_valid_pred)

                print('results for test set')
                y_test_pred = regressor.predict(X_test)
                test_rmse = return_rmse(y_test,y_test_pred)


                one_df = pd.DataFrame([[model, train_rmse, valid_rmse, test_rmse, train_time, batch, lr, loss_function ]],
                              columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'Batch Size', 
                                   'Learning Rate', 'Loss Function'])
                rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../2LSTM_grid_search_part3.csv')

            

training start for LSTM_LSTM
Epoch 1/50


INFO:plaidml:Analyzing Ops: 2198 of 2743 operations complete


252416/252438 [============================>.] - ETA: 0s - loss: 0.0036

INFO:plaidml:Analyzing Ops: 2102 of 2743 operations complete


252438/252438 [==============================] - 215s 851us/step - loss: 0.0036
Epoch 2/50
252438/252438 [==============================] - 189s 748us/step - loss: 0.0024
Epoch 3/50
252438/252438 [==============================] - 195s 773us/step - loss: 0.0023
Epoch 4/50
252438/252438 [==============================] - 189s 748us/step - loss: 0.0022
Epoch 5/50
252438/252438 [==============================] - 171s 678us/step - loss: 0.0022
Epoch 6/50
252438/252438 [==============================] - 194s 767us/step - loss: 0.0021
Epoch 7/50
252438/252438 [==============================] - 191s 758us/step - loss: 0.0021
Epoch 8/50
252438/252438 [==============================] - 191s 757us/step - loss: 0.0021
Epoch 9/50
252438/252438 [==============================] - 191s 756us/step - loss: 0.0020
Epoch 10/50
252438/252438 [==============================] - 170s 672us/step - loss: 0.0020
Epoch 11/50
252438/252438 [==============================] - 191s 756us/step - loss: 0.0020
Epoch 12

...

/Users/Harry/Documents/apple/env/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_LSTM
Epoch 1/50
252438/252438 [==============================] - 136s 538us/step - loss: 0.0217
Epoch 2/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0062
Epoch 3/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0042
Epoch 4/50
252438/252438 [==============================] - 135s 537us/step - loss: 0.0033
Epoch 5/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0031
Epoch 6/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0029
Epoch 7/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0028
Epoch 8/50
252438/252438 [==============================] - 135s 536us/step - loss: 0.0027
Epoch 9/50
252438/252438 [==============================] - 136s 537us/step - loss: 0.0027
Epoch 10/50
252438/252438 [==============================] - 135s 536us/step - loss: 0.0026
Epoch 11/50
252438/252438 [==============================] -

/Users/Harry/Documents/apple/env/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



training start for LSTM_LSTM
Epoch 1/50
252438/252438 [==============================] - 127s 504us/step - loss: 0.0537
Epoch 2/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0370
Epoch 3/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0350
Epoch 4/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0338
Epoch 5/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0329
Epoch 6/50
252438/252438 [==============================] - 127s 503us/step - loss: 0.0321
Epoch 7/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0314
Epoch 8/50
252438/252438 [==============================] - 127s 502us/step - loss: 0.0308
Epoch 9/50
252438/252438 [==============================] - 127s 503us/step - loss: 0.0303
Epoch 10/50
252438/252438 [==============================] - 127s 503us/step - loss: 0.0300
Epoch 11/50
252438/252438 [==============================] -

/Users/Harry/Documents/apple/env/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

